In [36]:
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
# Importing Gensim for LDA
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# PLotting tools 

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('C:/Users/abhis/OneDrive/Documents/GitHub/PeopleTail_NLP/src/reviews.csv')
df.count(), df.columns


(body        1873
 watson      1873
 products    1824
 user        1873
 dtype: int64, Index(['body', 'watson', 'products', 'user'], dtype='object'))

In [3]:
df['reviewtext'] = df['body'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

In [4]:
df.head()

,body,watson,products,user,reviewtext
0,"<p class=""MsoNormal"">Ah spring time is on the ...","{""usage"":{""text_units"":1,""text_characters"":230...",True Moringa - All Purpose Moringa Body Oil wi...,5,Ah spring time is on the horizon. Why not give...
1,"<p dir=""ltr"" style=""line-height: 1.38; margin-...","{""usage"":{""text_units"":1,""text_characters"":280...",BioFinest Pomegranate Seed Organic Oil - 100% ...,5,Feeling like you need a kick in the pants to g...
2,"<p class=""MsoNormal"">Oral health is key for ov...","{""usage"":{""text_units"":1,""text_characters"":133...","Lakanto Monkfruit Sweetener, Golden, 8.29 Ounc...",5,Oral health is key for overall health. Did yo...
3,"<p class=""MsoNormal"">The benefits of adding le...","{""usage"":{""text_units"":1,""text_characters"":117...",NaN,5,The benefits of adding lemon to our drinking w...
4,"<p class=""MsoNormal"">Our diets today, unfortun...","{""usage"":{""text_units"":1,""text_characters"":210...",Gelatinized Maca Root Powder From Peru - Certi...,5,"Our diets today, unfortunately have many hidde..."


# Preprocess the data and normalize it

In [22]:
stop_words = set(stopwords.words('english'))

# Convert to list
data = df.reviewtext.values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

['Ah spring time is on the horizon. Why not give your mane a spring detox and make over? Dry and cold winds can deplete your hair and scalp of moisture. Overtime your locks can become dry and brittle. Try this easy and pampering recommendation to protect your mane to look lustrous and feel silky smooth again. Apply True Moringa - All Purpose Moringa Body Oil with Peppermint, to your hair and scalp, leave in hair for 30 minutes or wrap your hair in a towel and leave in overnight. Moringa oil does not clog scalp pores, strengthens hair follicles and combats frizz. Keep it in your hair overnight into the next day (really its not greasy at all?). Wash it out with a mild, sulphate free shampoo, like the very juicy ACURE Mega Moisture Shampoo and reveal an uber moisturized hair and scalp. Next give your scalp the sustained moisture lock it deserves. We love the organic hair mask for its triple threat recovery from the dry blustery cold air that shocked our scalp and hair during the winter mo

<>:7: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \s
<ipython-input-22-6db0ed0c89cc>:7: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [23]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

data_words = list(sent_to_words(data))

print(data_words[:1])


[['ah', 'spring', 'time', 'is', 'on', 'the', 'horizon', 'why', 'not', 'give', 'your', 'mane', 'spring', 'detox', 'and', 'make', 'over', 'dry', 'and', 'cold', 'winds', 'can', 'deplete', 'your', 'hair', 'and', 'scalp', 'of', 'moisture', 'overtime', 'your', 'locks', 'can', 'become', 'dry', 'and', 'brittle', 'try', 'this', 'easy', 'and', 'pampering', 'recommendation', 'to', 'protect', 'your', 'mane', 'to', 'look', 'lustrous', 'and', 'feel', 'silky', 'smooth', 'again', 'apply', 'true', 'moringa', 'all', 'purpose', 'moringa', 'body', 'oil', 'with', 'peppermint', 'to', 'your', 'hair', 'and', 'scalp', 'leave', 'in', 'hair', 'for', 'minutes', 'or', 'wrap', 'your', 'hair', 'in', 'towel', 'and', 'leave', 'in', 'overnight', 'moringa', 'oil', 'does', 'not', 'clog', 'scalp', 'pores', 'strengthens', 'hair', 'follicles', 'and', 'combats', 'frizz', 'keep', 'it', 'in', 'your', 'hair', 'overnight', 'into', 'the', 'next', 'day', 'really', 'its', 'not', 'greasy', 'at', 'all', 'wash', 'it', 'out', 'with', '

In [24]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [25]:
print(data_words_nostops[:1])

[['ah', 'spring', 'time', 'horizon', 'give', 'mane', 'spring', 'detox', 'make', 'dry', 'cold', 'winds', 'deplete', 'hair', 'scalp', 'moisture', 'overtime', 'locks', 'become', 'dry', 'brittle', 'try', 'easy', 'pampering', 'recommendation', 'protect', 'mane', 'look', 'lustrous', 'feel', 'silky', 'smooth', 'apply', 'true', 'moringa', 'purpose', 'moringa', 'body', 'oil', 'peppermint', 'hair', 'scalp', 'leave', 'hair', 'minutes', 'wrap', 'hair', 'towel', 'leave', 'overnight', 'moringa', 'oil', 'clog', 'scalp', 'pores', 'strengthens', 'hair', 'follicles', 'combats', 'frizz', 'keep', 'hair', 'overnight', 'next', 'day', 'really', 'greasy', 'wash', 'mild', 'sulphate', 'free', 'shampoo', 'like', 'juicy', 'acure', 'mega', 'moisture', 'shampoo', 'reveal', 'uber', 'moisturized', 'hair', 'scalp', 'next', 'give', 'scalp', 'sustained', 'moisture', 'lock', 'deserves', 'love', 'organic', 'hair', 'mask', 'triple', 'threat', 'recovery', 'dry', 'blustery', 'cold', 'air', 'shocked', 'scalp', 'hair', 'winter

In [31]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [38]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

nlp = spacy.load('en', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [39]:
print(data_lemmatized[:1])

[['spring', 'give', 'spring', 'detox', 'make', 'dry', 'cold', 'wind', 'deplete', 'hair', 'overtime', 'lock', 'become', 'dry', 'brittle', 'try', 'easy', 'pamper', 'recommendation', 'mane', 'look', 'lustrous', 'feel', 'apply', 'true', 'moringa', 'purpose', 'body', 'oil', 'peppermint', 'hair', 'scalp', 'leave', 'hair', 'minute', 'wrap', 'hair', 'towel', 'leave', 'overnight', 'pore', 'strengthen', 'hair', 'follicle', 'combat', 'frizz', 'keep', 'hair', 'overnight', 'next', 'day', 'really', 'greasy', 'wash', 'mild', 'shampoo', 'acure', 'shampoo', 'moisturize', 'hair', 'next', 'give', 'sustain', 'moisture', 'lock', 'deserve', 'love', 'organic', 'hair', 'mask', 'triple', 'threat', 'recovery', 'dry', 'cold', 'shock', 'hair', 'apply', 'time', 'week', 'hair', 'milk', 'rise', 'ingredient', 'rise', 'water', 'balance', 'moisture', 'level', 'seed', 'oil', 'repair', 'nourish', 'hair', 'gently', 'cleanse', 'away', 'excess', 'harmful', 'product', 'diyer', 'make', 'simple', 'hair', 'mask', 'food', 'ingre

# Sentiment Analysis

In [40]:
analyzer = SentimentIntensityAnalyzer()

In [41]:
sentiment = df['reviewtext'].apply(lambda x: analyzer.polarity_scores(x))

In [42]:
df = pd.concat([df,sentiment.apply(pd.Series)],1)

In [43]:
df.head()

,body,watson,products,user,reviewtext,reviewtext_token,text_lemmatized,reviewtext_new,neg,neu,pos,compound
0,"<p class=""MsoNormal"">Ah spring time is on the ...","{""usage"":{""text_units"":1,""text_characters"":230...",True Moringa - All Purpose Moringa Body Oil wi...,5,Ah spring time is on the horizon. Why not give...,"[ah, spring, time, horizon, why, give, mane, s...","[ah, spring, time, horizon, why, give, mane, s...","[[], [], [], [], [], [], [], [], [], [], [], [...",0.042,0.846,0.112,0.9860
1,"<p dir=""ltr"" style=""line-height: 1.38; margin-...","{""usage"":{""text_units"":1,""text_characters"":280...",BioFinest Pomegranate Seed Organic Oil - 100% ...,5,Feeling like you need a kick in the pants to g...,"[feeling, like, need, kick, pants, get, stuper...","[feeling, like, need, kick, pant, get, stuper,...","[[], [], [], [], [], [], [], [], [], [], [], [...",0.072,0.731,0.197,0.9974
2,"<p class=""MsoNormal"">Oral health is key for ov...","{""usage"":{""text_units"":1,""text_characters"":133...","Lakanto Monkfruit Sweetener, Golden, 8.29 Ounc...",5,Oral health is key for overall health. Did yo...,"[oral, health, key, overall, health, did, know...","[oral, health, key, overall, health, did, know...","[[], [], [], [], [], [], [], [], [], [], [], [...",0.071,0.787,0.142,0.9473
3,"<p class=""MsoNormal"">The benefits of adding le...","{""usage"":{""text_units"":1,""text_characters"":117...",NaN,5,The benefits of adding lemon to our drinking w...,"[the, benefits, adding, lemon, drinking, water...","[the, benefit, adding, lemon, drinking, water,...","[[], [], [], [], [], [], [], [], [], [], [], [...",0.015,0.762,0.224,0.9924
4,"<p class=""MsoNormal"">Our diets today, unfortun...","{""usage"":{""text_units"":1,""text_characters"":210...",Gelatinized Maca Root Powder From Peru - Certi...,5,"Our diets today, unfortunately have many hidde...","[our, diets, today, unfortunately, many, hidde...","[our, diet, today, unfortunately, many, hidden...","[[], [], [], [], [], [], [], [], [], [], [], [...",0.034,0.827,0.138,0.9833


# LDA topic modelling

In [44]:
id2word = corpora.Dictionary(df['text_lemmatized'])

In [46]:
texts = data_lemmatized

In [47]:
corpus = [id2word.doc2bow(text) for text in texts]

In [48]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [49]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\abhis\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.395049  0.059412       1        1  43.105232
3      0.318577 -0.078081       2        1  21.643648
12     0.289436  0.169331       3        1  10.723032
8      0.198660  0.174936       4        1   8.463465
16     0.220293 -0.364093       5        1   6.548464
6     -0.009933  0.022358       6        1   2.400984
13    -0.077817 -0.006558       7        1   1.085731
18    -0.066115 -0.004253       8        1   1.005928
1     -0.082165  0.013151       9        1   0.968111
2     -0.095807 -0.006415      10        1   0.776073
5     -0.105864 -0.000732      11        1   0.731104
0     -0.101741 -0.001597      12        1   0.534162
11    -0.108125  0.000579      13        1   0.525758
19    -0.108903  0.000630      14        1   0.499519
10    -0.110721  0.001562      15        1   0.340935
14    -0.111926  0.002684      16        1   0.245052
4     -0.111376  0.004319      17        1   0.209930
15    -0.111612  0.003601      18        1   0.127700
17    -0.110260  0.004460      19        1   0.055181
9     -0.109653  0.004706      20        1   0.009989, topic_info=     Category         Freq          Term        Total  loglift  logprob
301   Default  1388.000000          skin  1388.000000  30.0000  30.0000
56    Default   591.000000          hair   591.000000  29.0000  29.0000
487   Default  2882.000000           use  2882.000000  28.0000  28.0000
174   Default  1105.000000           add  1105.000000  27.0000  27.0000
96    Default  1066.000000           oil  1066.000000  26.0000  26.0000
110   Default  2010.000000       product  2010.000000  25.0000  25.0000
285   Default   751.000000       protein   751.000000  24.0000  24.0000
161   Default   867.000000         water   867.000000  23.0000  23.0000
313   Default   574.000000         sugar   574.000000  22.0000  22.0000
16    Default  1241.000000          body  1241.000000  21.0000  21.0000
3234  Default   315.000000      collagen   315.000000  20.0000  20.0000
74    Default  1484.000000          make  1484.000000  19.0000  19.0000
382   Default   795.000000        health   795.000000  18.0000  18.0000
230   Default  1375.000000          help  1375.000000  17.0000  17.0000
72    Default  1368.000000          love  1368.000000  16.0000  16.0000
421   Default   736.000000         taste   736.000000  15.0000  15.0000
158   Default   830.000000       vitamin   830.000000  14.0000  14.0000
46    Default   585.000000          food   585.000000  13.0000  13.0000
154   Default  1189.000000           try  1189.000000  12.0000  12.0000
279   Default   369.000000        powder   369.000000  11.0000  11.0000
89    Default   916.000000       natural   916.000000  10.0000  10.0000
215   Default   385.000000          face   385.000000   9.0000   9.0000
523   Default   342.000000         enjoy   342.000000   8.0000   8.0000
1192  Default   356.000000       flavour   356.000000   7.0000   7.0000
23    Default   249.000000          cold   249.000000   6.0000   6.0000
481   Default   541.000000    supplement   541.000000   5.0000   5.0000
35    Default   370.000000           dry   370.000000   4.0000   4.0000
160   Default   357.000000          wash   357.000000   3.0000   3.0000
562   Default   317.000000        recipe   317.000000   2.0000   2.0000
693   Default   175.000000        stress   175.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
6028  Topic20     0.001048  biochemistry     0.859050   2.5026  -9.5351
4607  Topic20     0.001048       chapter     0.859049   2.5026  -9.5351
4618  Topic20     0.001048     elemental     0.859050   2.5026  -9.5351
4617  Topic20     0.001048    digestable     0.859049   2.5026  -9.5351
4616  Topic20     0.001048         davis     0.859049   2.5026  -9.5351
4615  Topic20     0.001048        cooked     0.859060   2.5026  -9.5351
4614  Topic20     0.001048       